In [51]:
import pandas as pd
import json
import time
import re
import dotenv
import os

In [52]:
df = pd.read_csv("yelp.csv") 

In [53]:
df.head()

In [54]:
df.info()

In [55]:
df.duplicated().sum()

In [56]:
df['stars'].value_counts()

In [57]:
df_task = df[["text", "stars"]].dropna()
df_task = df_task.sample(n=200, random_state=42).reset_index(drop=True)
df_task.head()

In [58]:
dotenv.load_dotenv()
from openai import OpenAI
import os
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY") 


## Prompt Engineering Strategies

To satisfy the task requirements, we have designed three distinct prompting approaches to evaluate performance:

1. **Prompt V1 (Baseline)**: A direct, **zero-shot** instruction. This represents the simplest approach, establishing a baseline to see how well the model understands the task with minimal guidance.
2. **Prompt V2 (Few-Shot)**: Adds **context and examples** (one negative, one mixed, one positive). This strategy aims to "calibrate" the model to the specific nuances of Yelp reviews and the JSON output format, theoretically improving consistency.
3. **Prompt V3 (Strict/Reasoning)**: Implements constraints and a **Chain-of-Thought** style directive. By forcing the model to internally identify aspects (service, food) before rating, we aim to better handle complex reviews that might have conflicting sentiments.

In [59]:
prompt_v1  = """
You are a sentiment analysis API.

Classify the following Yelp review into a star rating from 1 to 5.

Return ONLY a valid JSON object in the following format:
{{
  "predicted_stars": <integer>,
  "explanation": "<short reasoning>"
}}

Do NOT include markdown, backticks, or any text outside the JSON.

Review:
{review_text}
"""



In [60]:
prompt_v2 = """
Determine the star rating (1 to 5) for the Yelp review below.

Use the following examples as a guide:

Input: "Horrible service, waiter was rude and food was cold."
Output: {{ "predicted_stars": 1, "explanation": "Negative complaints about both service and food quality." }}

Input: "It was okay. The burger was good but the fries were soggy."
Output: {{ "predicted_stars": 3, "explanation": "Mixed experience with both positive and negative elements." }}

Input: "Absolutely loved it! Best pasta I've ever had."
Output: {{ "predicted_stars": 5, "explanation": "Enthusiastic praise with no complaints." }}

Now classify the following review.

Return ONLY a valid JSON object.
Do NOT include markdown, backticks, or any text outside the JSON.

Format:
{{
  "predicted_stars": <integer>,
  "explanation": "<short reasoning>"
}}

Review:
{review_text}
"""



In [61]:
prompt_v3 = """
You are a strict classification system for Yelp reviews.

Internally:
- Identify key aspects (service, food, price, etc.)
- Assess sentiment for each aspect
- Weigh positives and negatives to select the final rating

Do NOT include internal reasoning in the output.

Return ONLY valid JSON in exactly this format:
{{
  "predicted_stars": <integer between 1 and 5>,
  "explanation": "<brief justification>"
}}

IMPORTANT:
- Do NOT include markdown, backticks, or extra text.

Review:
{review_text}
"""



In [62]:
def build_prompt(prompt_template, review_text):
    return prompt_template.format(review_text=review_text)

In [63]:
client_or = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

def call_llm(prompt):
    response = client_or.chat.completions.create(
        model="meta-llama/llama-3.1-8b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content


In [64]:
def parse_response(response_text):
    # Try to find JSON if wrapped in markdown
    match = re.search(r'{.*}', response_text, re.DOTALL)
    json_str = match.group(0) if match else response_text

    try:
        parsed = json.loads(json_str)
        if "predicted_stars" in parsed:
            return parsed, True
    except json.JSONDecodeError:
        pass
    
    return None, False


In [ ]:
def evaluate_prompt(prompt_template, df_eval):
    records = []

    for _, row in df_eval.iterrows():
        review_text = row['text']
        actual_stars = row["stars"]

        prompt = build_prompt(prompt_template, review_text)
        raw_output = call_llm(prompt) 

        parsed, json_valid = parse_response(raw_output) if raw_output else (None, False)
        predicted_stars = parsed["predicted_stars"] if json_valid else None

        records.append({
            "actual_stars": actual_stars,
            "predicted_stars": predicted_stars,
            "json_valid": json_valid,
            "raw_output": raw_output
        })

        time.sleep(2) 
    
    df_results = pd.DataFrame(records)
    
    # Calculate metrics safely
    valid_results = df_results[df_results['json_valid'] == True]
    accuracy = (valid_results['actual_stars'] == valid_results['predicted_stars']).mean() if len(valid_results) > 0 else 0
    json_validity_rate = df_results['json_valid'].mean() if len(df_results) > 0 else 0

    print(f"\nEvaluation Complete: Accuracy={accuracy:.3f}, JSON Validity Rate={json_validity_rate:.3f}")

    return df_results, accuracy, json_validity_rate


In [ ]:
results_v1, acc_v1, json_v1 = evaluate_prompt(prompt_v1, df_task)
results_v2, acc_v2, json_v2 = evaluate_prompt(prompt_v2, df_task)
results_v3, acc_v3, json_v3 = evaluate_prompt(prompt_v3, df_task)


In [ ]:
comparison_table = pd.DataFrame([
    {"prompt": "v1_baseline", "accuracy": acc_v1, "json_validity": json_v1},
    {"prompt": "v2_few_shot", "accuracy": acc_v2, "json_validity": json_v2},
    {"prompt": "v3_strict", "accuracy": acc_v3, "json_validity": json_v3},
])

comparison_table


# Discussion of Results and Trade-offs

## Results Analysis
All three prompts achieved 100% JSON validity, verifying the robustness of the implementation.

- **Accuracy**: **`v3_strict` performed best (0.640)**, finding a notable advantage over `v1_baseline` (0.620) and `v2_few_shot` (0.595).
- **Observation**: The strict prompting strategy, which encourages internal component-based analysis (weighing negatives vs positives), proved superior here. It likely helped the model navigate mixed-sentiment reviews where a simple "gut feel" classification (V1) or a patterned few-shot approach (V2) failed.

## Trade-offs
| Prompt Type | Pros | Cons |
|------------|------|------|
| **V1 (Baseline)** | Low token cost, simple. Good baseline performance. | Lacks nuance for complex cases. |
| **V2 (Few-Shot)** | Structured. Helps with format adherence. | Highest token cost. Examples can bias the model if not perfectly representative. |
| **V3 (Strict/CoT)** | **Highest Accuracy**. Best at handling nuance and mixed feelings. | More complex prompt structure. |

**Conclusion**: For this dataset, the **V3 Strict** prompt is the clear winner, justifying the slight complexity increase with superior accuracy.